In [1]:
# https://qwen.readthedocs.io/en/latest/framework/LlamaIndex.html

import torch
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Set prompt template for generation (optional)
from llama_index.core import PromptTemplate


def completion_to_prompt(completion):
    return f"<|im_start|>system\n<|im_end|>\n<|im_start|>user\n{completion}<|im_end|>\n<|im_start|>assistant\n"


def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == "system":
            prompt += f"<|im_start|>system\n{message.content}<|im_end|>\n"
        elif message.role == "user":
            prompt += f"<|im_start|>user\n{message.content}<|im_end|>\n"
        elif message.role == "assistant":
            prompt += f"<|im_start|>assistant\n{message.content}<|im_end|>\n"

    if not prompt.startswith("<|im_start|>system"):
        prompt = "<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n" + prompt

    prompt = prompt + "<|im_start|>assistant\n"

    return prompt


def setup(model, tokenizer, embed_model, input_dir = None, input_files = None):
    # Set Qwen2.5 as the language model and set generation config
    Settings.llm = HuggingFaceLLM(
        model_name=model, # "Qwen/Qwen3-235B-A22B-Instruct-2507",  # "Qwen/Qwen2.5-7B-Instruct",
        tokenizer_name=tokenizer, # "Qwen/Qwen3-235B-A22B-Instruct-2507",  # "Qwen/Qwen2.5-7B-Instruct",
        context_window=30000,
        max_new_tokens=2000,
        generate_kwargs={
            "temperature": 0.7,
            "top_k": 20,
            "top_p": 0.8,
        },  # {"temperature": 0.7, "top_k": 50, "top_p": 0.95},
        messages_to_prompt=messages_to_prompt,
        completion_to_prompt=completion_to_prompt,
        device_map="auto",
    )

    # Set embedding model
    Settings.embed_model = HuggingFaceEmbedding(
        model_name=embed_model, # "BAAI/bge-multilingual-gemma2"  # "BAAI/bge-base-en-v1.5"
    )

    # Set the size of the text chunk for retrieval
    Settings.transformations = [SentenceSplitter(chunk_size=1024)]

    from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

    if input_dir is not None:
        documents = SimpleDirectoryReader(
            input_dir=input_dir
        ).load_data()
    elif input_files is not None:
        documents = SimpleDirectoryReader(
            input_files=input_files
        ).load_data()
        
    Settings.index = VectorStoreIndex.from_documents(
        documents,
        embed_model=Settings.embed_model,
        transformations=Settings.transformations,
    )

def query(query_text):
    query_engine = Settings.index.as_query_engine()
    response = query_engine.query(query_text).response
    print(response)
    #return response

In [2]:
#setup(
#    model = "Qwen/Qwen2.5-7B-Instruct",
#    tokenizer = "Qwen/Qwen2.5-7B-Instruct",
#    embed_model = "BAAI/bge-multilingual-gemma2",
#    input_files = ["../document/saiyuuki.txt"])

#setup(
#    model = "Qwen/Qwen3-235B-A22B-Instruct-2507",
#    tokenizer = "Qwen/Qwen3-235B-A22B-Instruct-2507",
#    embed_model = "BAAI/bge-multilingual-gemma2",
#    input_files = ["../document/saiyuuki.txt"])

setup(
    model = "Qwen/Qwen3-30B-A3B-Instruct-2507",
    tokenizer = "Qwen/Qwen3-30B-A3B-Instruct-2507",
    embed_model = "BAAI/bge-multilingual-gemma2",
    input_files = ["../document/saiyuuki.txt"])


Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/329 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [3]:
query("悟空の飛行方法を教えてください。道具を使用して飛行しますか？")

悟空の飛行方法は、道具を使用せず、自身の法術（呪文）によって実現しています。彼は「念声咒語」を唱え、風雲を駆使して空中を飛行します。文中では「好猴王念声咒语驾阵狂风云头落下」と記されており、これは悟空が呪文を唱えることで風雲を操り、空中を移動していることを示しています。したがって、道具ではなく、自身の修行によって得た能力で飛行しています。


In [4]:
query("沙悟浄は人間でしょうか？")

沙悟净（さごじょう）は人間ではありません。彼は「卷帘大将（けんれんたいしょう）」という天界の神々しい存在が、天で罪を犯して流沙河（りゅうさが）に堕ち、形を失って妖魔として生きている存在です。彼は観音菩薩によって教化され、法名を「沙悟净」として「沙」を姓として受け、取経の旅に従うことを誓います。したがって、彼は人間ではなく、元は天界の将軍であり、現在は取経の旅に参加する修行者（弟子）として登場しています。


In [5]:
query("物語の最後で悟空はどうなりますか？")

物語の最後で悟空（孫悟空）は、唐僧（三蔵法師）と共に東土中国へ帰還し、五聖（五人の聖者）として成仏します。彼は長い修行と試練を経て、本来の姿を悟り、真の悟りを得ます。最終的に、彼は仏の位に昇り、法を広める使命を果たすとともに、仲間たちと共に「五聖成真」となり、永遠の悟りを得ます。
